In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import string
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.stem.porter import PorterStemmer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
basketball = pd.read_csv("/content/drive/My Drive/Tolu/tolufilebasketball.csv")


tennis = pd.read_csv("/content/drive/My Drive/Tolu/tolufiletennis.csv")

cricket = pd.read_csv("/content/drive/My Drive/Tolu/tolufilecricket.csv")

In [ ]:
headlines = pd.concat([basketball, tennis, cricket], axis=0)

In [ ]:
headlines = headlines.sample(frac = 1)

In [ ]:
headlines

,title,value
1697,T20 World Cup 2022: Top 5 matches of this edit...,2
4947,"James, Davis help Lakers win for first time th...",1
4867,"Tipton scores 21, Bellarmine holds off Louisvi...",1
2299,Suryakumar Yadav first Indian to achieve huge ...,2
4561,TIPOFF: These teams seek NCAA bids they didn't...,1
...,...,...
2315,Zimbabwe vs India live stream — how to watch t...,2
1333,‘Glow in the Dark' tennis party is a special n...,0
4569,"Kiser, Phelia combine for 46, No. 25 Michigan ...",1
405,Matthew Hoggard to appeal decision to hold Yor...,2


In [ ]:
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", str(text))


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)
def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)
def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)
  

In [ ]:
headlines["title"] = headlines.title.map(lambda x: remove_URL(x))
headlines["title"] = headlines.title.map(lambda x: remove_html(x))
headlines["title"] = headlines.title.map(lambda x: remove_emoji(x))
headlines["title"] = headlines.title.map(lambda x: remove_punct(x))

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)
headlines["title"] =headlines["title"].map(remove_stopwords)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
headlines

,title,value
1697,t20 world cup 2022 top 5 matches edition thus far,2
4947,james davis help lakers win first time season,1
4867,tipton scores 21 bellarmine holds louisville 6766,1
2299,suryakumar yadav first indian achieve huge t20...,2
4561,tipoff teams seek ncaa bids didnt get last year,1
...,...,...
2315,zimbabwe vs india live stream — watch t20 worl...,2
1333,‘glow dark tennis party special night delray b...,0
4569,kiser phelia combine 46 25 michigan women beat...,1
405,matthew hoggard appeal decision hold yorkshire...,2


In [ ]:
vocabulary_size = 10000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(headlines['title'])
sequences = tokenizer.texts_to_sequences(headlines['title'])
X_CNN = pad_sequences(sequences, maxlen=50)

In [ ]:
## Split the data into train and test
Y_CNN = headlines["value"]
X_train_CNN, X_test_CNN, Y_train_CNN, Y_test_CNN = train_test_split(X_CNN, 
                       Y_CNN, test_size=0.3, random_state=7)

In [ ]:
X_test_CNN.shape

(3000, 50)

In [ ]:
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
max_features = 10000
embedding_dim = 300
sequence_length = 50
def create_model(input_length=50):

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(max_features, embedding_dim, input_length=sequence_length)) 

    model.add(tf.keras.layers.Conv1D(128,3, activation='relu', padding="same")) 

    model.add(tf.keras.layers.GlobalMaxPooling1D())

    model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(3, activation='sigmoid'))

    model.summary()

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model_CNN = KerasClassifier(build_fn=create_model, epochs=3, verbose=1, validation_split=0.4)
model_CNN.fit(X_train_CNN, Y_train_CNN)
    

<ipython-input-17-a7461e6d1f88>:26: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_CNN = KerasClassifier(build_fn=create_model, epochs=3, verbose=1, validation_split=0.4)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 300)           3000000   
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 128)           115328    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 3,115,715
Trainable params: 3,115,715
Non-trainable params: 0
____________________________________________

In [ ]:
# performance metrics
import sklearn
from sklearn.metrics import classification_report
y_pred = model_CNN.predict(X_test_CNN)
report = classification_report(Y_test_CNN, y_pred.round())
print(report)

94/94 [==============================] - 1s 12ms/step
              precision    recall  f1-score   support

           0       0.93      0.92      0.93       739
           1       0.96      0.98      0.97      1492
           2       0.96      0.93      0.94       769

    accuracy                           0.95      3000
   macro avg       0.95      0.94      0.95      3000
weighted avg       0.95      0.95      0.95      3000



In [ ]:
import matplotlib.pyplot as plt
plt.plot(model_CNN.history['loss'])
plt.plot(model_CNN.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

AttributeError: ignored